In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn

# import pytorch_lightning as pl
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchmetrics as TM
# pl.utilities.seed.seed_everything(seed=42)
import numpy as np
import pandas as pd

import sys, os
source_path = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(source_path)

source_path = os.path.join(os.getcwd(), os.pardir, 'preprocessing')
sys.path.append(source_path)

source_path = os.path.join(os.getcwd(), os.pardir, 'metrics')
sys.path.append(source_path)

source_path = os.path.join(os.getcwd(), os.pardir, 'predict')
sys.path.append(source_path)

import matplotlib.pyplot as plt

from dl import NeuralNetwork, Trainer
from preprocess import (
    show_df, 
    date_features, 
    preprocess, 
    ToTorch, 
    get_loader, 
    get_predict_loader,
    ts_split,
    cont_cat_split,
    dataloader_by_stock,
    get_data,
    dataloader_test_by_stock
)
from metrics.metrics import calc_spread_return_sharpe
from predict import run_pred_step
print(torch.__version__)

import pickle

import warnings
warnings.filterwarnings("ignore")

C:\Users\gilbe\anaconda3\envs\tokyo\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.11.0


# Get data

In [2]:
FOLDER = 'example_test_files'
BATCH_SIZE = 512

df = get_data(folder=FOLDER)
# df['Target'] = df['Close'].pct_change()
# df = df.sort_values(by=['RowId', 'Date'])
print(df.head())
"""
SUBTRACT:
*) 3 FROM CONTINUOUS COLS BECAUSE OF POOLING
*) 1 FROM CAT_FEATURES TO MAKE MATRICES MATCH AFTER TORCH.CAT
"""
CONT_COLS=['Close', 'Open', 'High', 'Low', 'MarketCapitalization',         
           # 'NetSales', 'EquityToAssetRatio', 'TotalAssets', 'Profit', 
           # 'OperatingProfit', 'EarningsPerShare', 'Equity', 
           # 'BookValuePerShare', 'ResultDividendPerShare1stQuarter', 
           # 'ResultDividendPerShare2ndQuarter', 'ResultDividendPerShare3rdQuarter',
           # 'ResultDividendPerShareFiscalYearEnd', 'ResultDividendPerShareAnnual'
          ]
TS_IN_FEATURES = len(CONT_COLS)
CAT_FEATURES = 4 + 4 - 1 # TEXT_COLS = ['Section/Products', '33SectorName', '17SectorName', 'Universe0']
print('CAT_FEATURES:', CAT_FEATURES)
EMBEDDING_DIM = 300
NO_EMBEDDING = 2000 #2 * len(df_train_cat)

train_df.head(10):
            Section/Products  33SectorName  17SectorName  Universe0  \
Date                                                                  
2021-12-06                 0             6             8          0   
2021-12-07                 0             6             8          0   
2021-12-06                 0             6             8          0   
2021-12-07                 0             6             8          0   
2021-12-06                 0             6             8          0   
2021-12-07                 0             6             8          0   
2021-12-06                 0             6             8          0   
2021-12-07                 0             6             8          0   
2021-12-06                 0             6             8          0   
2021-12-07                 0             6             8          0   

            MarketCapitalization  SecuritiesCode          RowId    Open  \
Date                                                 

In [3]:
df['SecuritiesCode'].unique()

array([1301, 1332, 1333, ..., 9993, 9994, 9997], dtype=int64)

In [4]:
df[df['SecuritiesCode'] == 1301]

,Section/Products,33SectorName,17SectorName,Universe0,MarketCapitalization,SecuritiesCode,RowId,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag
Date,,,,,,,,,,,,,,,
2021-12-06,0,6,8,0,3.365911e+10,1301,20211206_1301,2982.0,2982.0,2965.0,2971.0,8900,1.0,NaN,False
2021-12-07,0,6,8,0,3.365911e+10,1301,20211207_1301,2998.0,3065.0,2990.0,3065.0,19100,1.0,NaN,False


In [5]:
df[df['SecuritiesCode'] == 1301].shape

(2, 15)

In [13]:

import copy
stocks = df['SecuritiesCode'].unique()

# model = create_model()
# trainer = Trainer(model, optimizer_name='rmsprop', lr=1.3333e-4, weight_decay=weight_decay)
# torch.autograd.set_detect_anomaly(True)
# trained_model = torch.load('./trained_model.pt')


trained_model = NeuralNetwork(
        in_features=TS_IN_FEATURES - 3, 
        units=128,
        out_features=1, 
        categorical_dim=CAT_FEATURES,
        no_embedding=NO_EMBEDDING, 
        emb_dim=EMBEDDING_DIM,
        n_blocks=4,
        n_stacks=4,
        dropout=0.1,
        pooling_sizes=3
    )

# trained_model.load_state_dict(torch.load('./trained_model.pt'))

with open('./scaler_dict.pkl', 'rb') as f:
    scaler_dict = pickle.load(f)

    
pred_df = pd.DataFrame()
for stock in stocks:
    # teast_dataloader = None
    # if count > 500:
    #     break
    # try:
    print(f'Run prediction for stock: {stock}')
    # transformer = scaler_dict[stock]
    test_dataloader, idx = dataloader_test_by_stock(
        df, 
        stock, 
        # transformer=transformer,
        batch_size=BATCH_SIZE, 
        continous_cols=CONT_COLS,
        target_col=None
    )
    print('idx:', idx)
    pred_list = run_pred_step(test_dataloader, trained_model, x_cat=True, target=False)
    print('pred_list:', pred_list)
    df_pred = pd.DataFrame({'Values': pred_list, 'SecuritiesCoed': stock}, index=[idx[-1]])
    df_pred.index.rename('Date', inplace=True)
    pred_df = pd.concat([pred_df, df_pred]) #.pivot()
    print('pred_df')
    print(pred_df[:5])
    # pred_df[stock] = pred_list
    print()
    # count += 1
#     except Exception as e:
#         print(f'Exception {e}')
        

# pred_df = pd.DataFrame.from_dict(pred_df)

    

Run prediction for stock: 1301
continuos shape: (2, 10)  categorical shape: (2, 7)
Notice that the transformer is None.
idx: DatetimeIndex(['2021-12-06', '2021-12-07'], dtype='datetime64[ns]', name='Date', freq=None)
Test predictions:
-0.06420749 ()
pred_list: [array(-0.06420749, dtype=float32)]
pred_df
                 Values  SecuritiesCoed
Date                                   
2021-12-07  -0.06420749            1301

Run prediction for stock: 1332
continuos shape: (2, 10)  categorical shape: (2, 7)
Notice that the transformer is None.
idx: DatetimeIndex(['2021-12-06', '2021-12-07'], dtype='datetime64[ns]', name='Date', freq=None)
Test predictions:
-0.06404488 ()
pred_list: [array(-0.06404488, dtype=float32)]
pred_df
                 Values  SecuritiesCoed
Date                                   
2021-12-07  -0.06420749            1301
2021-12-07  -0.06404488            1332

Run prediction for stock: 1333
continuos shape: (2, 10)  categorical shape: (2, 7)
Notice that the transform

ValueError: Length of values (0) does not match length of index (1)

In [ ]:
pred_df.T

# Load model